# Current Blockchain ( 2nodes )

In [1]:
!rm -f /tmp/*.json
!rm -f /tmp/*.pb

In [4]:
!peer lifecycle chaincode querycommitted --channelID mychannel --name scray-invoice-example --output json

{
	"sequence": 1,
	"version": "1.0",
	"endorsement_plugin": "escc",
	"validation_plugin": "vscc",
	"validation_parameter": "EiAvQ2hhbm5lbC9BcHBsaWNhdGlvbi9FbmRvcnNlbWVudA==",
	"collections": {},
	"approvals": {
		"Org1MSP": true,
		"Org2MSP": true
	}
}


# init peer access (certificates,...)

In [3]:
import os
import json
import subprocess
import pandas as pd
 
pwd0 = "/home/jovyan/work/fabric-samples/"
pwd = "/home/jovyan/work/fabric-samples/test-network"

os.environ['PATH'] = "/home/jovyan/work/fabric-samples/bin:" + str(os.environ.get('PATH')) 
os.environ['CORE_PEER_TLS_ENABLED'] = "true"
os.environ['CORE_PEER_LOCALMSPID'] = "Org1MSP"
os.environ['CORE_PEER_TLS_ROOTCERT_FILE'] = pwd + "/organizations/peerOrganizations/org1.example.com/peers/peer0.org1.example.com/tls/ca.crt"
os.environ['CORE_PEER_MSPCONFIGPATH'] = pwd + "/organizations/peerOrganizations/org1.example.com/users/Admin@org1.example.com/msp"
os.environ['CORE_PEER_ADDRESS'] = "peer0.org1.example.com:7051"
os.environ['FABRIC_CFG_PATH'] = pwd0 + "/config/"

addr0 = 'peer0.org1.example.com:7050'
orderer = 'orderer.example.com'
ordererpem = pwd + '/organizations/ordererOrganizations/example.com/orderers/orderer.example.com/msp/tlscacerts/tlsca.example.com-cert.pem'
addr1 = 'peer0.org1.example.com:7051'
crt1 = pwd + '/organizations/peerOrganizations/org1.example.com/peers/peer0.org1.example.com/tls/ca.crt'
addr2 = 'peer0.org2.example.com:9051'
crt2 = pwd + '/organizations/peerOrganizations/org2.example.com/peers/peer0.org2.example.com/tls/ca.crt' 

msp1 = pwd + '/organizations/peerOrganizations/org1.example.com/users/Admin@org1.example.com/msp'
msp2 = pwd + '/organizations/peerOrganizations/org2.example.com/users/Admin@org2.example.com/msp'

In [5]:
os.environ['ORDERER_IP']="10.15.128.97"
os.environ['ORDERER_HOSTNAME']="orderer.example.com" 
os.environ['CHANNEL_NAME']="mychannel"
os.environ['ORG_ID']="OrgSally"

os.environ['CORE_PEER_LOCALMSPID']="Org1MSP"
os.environ['CORE_PEER_TLS_ROOTCERT_FILE']=crt1
os.environ['CORE_PEER_MSPCONFIGPATH']=msp1
os.environ['ORDERER_CA']=ordererpem

os.environ['CORE_PEER_ADDRESS'] = "peer0.org1.example.com:7051"

!peer channel fetch config /tmp/config_block.pb -o orderer.example.com:7050 -c $CHANNEL_NAME --tls --cafile $ORDERER_CA > /dev/null
!configtxlator proto_decode --input /tmp/config_block.pb --type common.Block | jq .data.data[0].payload.data.config > /tmp/config.json    

2020-12-08 09:56:43.914 UTC [channelCmd] InitCmdFactory -> INFO 001 Endorser and orderer connections initialized
2020-12-08 09:56:43.917 UTC [cli.common] readBlock -> INFO 002 Received block: 17
2020-12-08 09:56:43.918 UTC [channelCmd] fetch -> INFO 003 Retrieving last config block: 2
2020-12-08 09:56:43.922 UTC [cli.common] readBlock -> INFO 004 Received block: 2


In [6]:
# Upload CA cert
os.environ['SHARED_FS_HOST']="10.15.136.41:30080"
os.environ['SHARED_FS_USER']="scray"
os.environ['SHARED_FS_PW']="scray"
#!apk add curl

os.environ['NEW_ORG_NAME']="OrgSally"

!curl --user $SHARED_FS_USER:$SHARED_FS_PW -X MKCOL http://$SHARED_FS_HOST/ca
!curl --user $SHARED_FS_USER:$SHARED_FS_PW -T /home/jovyan/work/fabric-samples/test-network/organizations/ordererOrganizations/example.com/orderers/orderer.example.com/msp/tlscacerts/tlsca.example.com-cert.pem http://$SHARED_FS_HOST/ca/tlsca.example.com-cert.pem

# Get configuration of new peer
#curl --user 'scray:scray' http://${SHARED_FS_HOST}/newmemberrequests/$CHANNEL_NAME/${NEW_ORG_NAME}.json > new_member_org.json
!curl --user $SHARED_FS_USER:$SHARED_FS_PW  http://${SHARED_FS_HOST}/newmemberrequests/$CHANNEL_NAME/${NEW_ORG_NAME}.json > /tmp/new_member_org.json 
        
# Add org3 data to existing config
!jq -s ".[0] "*" {\"channel_group\":{\"groups\":{\"Application\":{\"groups\": {\"${NEW_ORG_NAME}MSP\":.[1]}}}}}" /tmp/config.json /tmp/new_member_org.json > /tmp/modified_config.json

!configtxlator proto_encode --input /tmp/config.json --type common.Config --output /tmp/config.pb
!configtxlator proto_encode --input /tmp/modified_config.json --type common.Config --output /tmp/modified_config.pb
!configtxlator compute_update --channel_id $CHANNEL_NAME --original /tmp/config.pb --updated /tmp/modified_config.pb --output /tmp/org3_update.pb
!configtxlator proto_decode --input /tmp/org3_update.pb --type common.ConfigUpdate | jq . > /tmp/org3_update.json
!echo '{"payload":{"header":{"channel_header":{"channel_id":"'$CHANNEL_NAME'", "type":2}},"data":{"config_update":'$(cat /tmp/org3_update.json)'}}}' | jq . > /tmp/org3_update_in_envelope.json
!configtxlator proto_encode --input /tmp/org3_update_in_envelope.json --type common.Envelope --output /tmp/org3_update_in_envelope.pb        

<!DOCTYPE HTML PUBLIC "-//IETF//DTD HTML 2.0//EN">
<html><head>
<title>301 Moved Permanently</title>
</head><body>
<h1>Moved Permanently</h1>
<p>The document has moved <a href="http://10.15.136.41:30080/ca/">here</a>.</p>
</body></html>
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 11435  100 11435    0     0  1861k      0 --:--:-- --:--:-- --:--:-- 1861k


sign and update

In [7]:
!peer channel signconfigtx -f /tmp/org3_update_in_envelope.pb

os.environ['CORE_PEER_LOCALMSPID']="Org2MSP"
os.environ['CORE_PEER_TLS_ROOTCERT_FILE']=crt2
os.environ['CORE_PEER_MSPCONFIGPATH']=msp2
os.environ['ORDERER_CA']=ordererpem
os.environ['CORE_PEER_ADDRESS'] = addr2

!peer channel update -f /tmp/org3_update_in_envelope.pb -c $CHANNEL_NAME -o orderer.example.com:7050 --tls --cafile $ORDERER_CA

2020-12-08 09:57:44.153 UTC [channelCmd] InitCmdFactory -> INFO 001 Endorser and orderer connections initialized
2020-12-08 09:57:44.967 UTC [channelCmd] InitCmdFactory -> INFO 001 Endorser and orderer connections initialized
2020-12-08 09:57:45.000 UTC [channelCmd] update -> INFO 002 Successfully submitted channel update


# network_install_and_approve_cc

In [8]:
#!pwd
#!ls /home/jovyan/work/scray/projects/invoice-hyperledger-fabric/scripts/example_network_install_and_approve_cc.sh
!cd /tmp
!/home/jovyan/work/scray/projects/invoice-hyperledger-fabric/scripts/example_network_install_and_approve_cc.sh 10.14.128.38 10.15.128.97 /home/jovyan/work/fabric-samples/test-network

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   450  100   450    0     0   3461      0 --:--:-- --:--:-- --:--:--  3461
2020-12-08 09:59:42.700 UTC [cli.lifecycle.chaincode] submitInstallProposal -> INFO 001 Installed remotely: response:<status:200 payload:"\nJbasic_1.0:5a294a12a1a89cd4eed3d4234fbc79f42eab2ac20cd176bc8ebbc07c597cd0ee\022\tbasic_1.0" > 
2020-12-08 09:59:42.700 UTC [cli.lifecycle.chaincode] submitInstallProposal -> INFO 002 Chaincode code package identifier: basic_1.0:5a294a12a1a89cd4eed3d4234fbc79f42eab2ac20cd176bc8ebbc07c597cd0ee
2020-12-08 09:59:42.793 UTC [cli.lifecycle.chaincode] submitInstallProposal -> INFO 001 Installed remotely: response:<status:200 payload:"\nJbasic_1.0:5a294a12a1a89cd4eed3d4234fbc79f42eab2ac20cd176bc8ebbc07c597cd0ee\022\tbasic_1.0" > 
2020-12-08 09:59:42.793 UTC [cli.lifecycle.chaincode] submitInstallProposal -> INFO 002 Chainco

# commit

In [9]:
os.environ['IP_CC_SERVICE']='10.14.128.38'
#Host where the example network is running
os.environ['IP_OF_EXAMPLE_NETWORK']='10.15.128.97'

!/home/jovyan/work/scray/projects/invoice-hyperledger-fabric/scripts/example_network_commit_cc.sh $IP_CC_SERVICE $IP_OF_EXAMPLE_NETWORK /home/jovyan/work/fabric-samples/test-network

2020-12-08 10:00:03.285 UTC [chaincodeCmd] ClientWait -> INFO 001 txid [265ea2c30331279df4776a68cd25b22a54b946659c5164488471e8bbb62f9424] committed with status (VALID) at peer0.org2.example.com:9051
2020-12-08 10:00:03.300 UTC [chaincodeCmd] ClientWait -> INFO 002 txid [265ea2c30331279df4776a68cd25b22a54b946659c5164488471e8bbb62f9424] committed with status (VALID) at peer0.org1.example.com:7051
2020-12-08 10:00:03.416 UTC [chaincodeCmd] chaincodeInvokeOrQuery -> INFO 001 Chaincode invoke successful. result: status:200 


# check network (approved chaincode)

In [10]:
!peer lifecycle chaincode querycommitted --channelID mychannel --name basic --output json

{
	"sequence": 1,
	"version": "1.0",
	"endorsement_plugin": "escc",
	"validation_plugin": "vscc",
	"validation_parameter": "EiAvQ2hhbm5lbC9BcHBsaWNhdGlvbi9FbmRvcnNlbWVudA==",
	"collections": {},
	"approvals": {
		"Org1MSP": true,
		"Org2MSP": true,
		"OrgSallyMSP": true
	}
}


In [11]:
!peer lifecycle chaincode querycommitted --channelID mychannel --name scray-invoice-example --output json

{
	"sequence": 1,
	"version": "1.0",
	"endorsement_plugin": "escc",
	"validation_plugin": "vscc",
	"validation_parameter": "EiAvQ2hhbm5lbC9BcHBsaWNhdGlvbi9FbmRvcnNlbWVudA==",
	"collections": {},
	"approvals": {
		"Org1MSP": true,
		"Org2MSP": true,
		"OrgSallyMSP": false
	}
}


# Test

In [12]:
def readAsset(asset='asset1'):
    callopt= '{\"function\":\"ReadAsset\",\"Args\":[' + '\"' + asset + '\"' + ']}'
    try:    
        callProcess  = subprocess.check_output(['peer', 'chaincode','query', '-C', 'mychannel', '-n', 'basic', '-c', callopt])
        return json.loads(str(callProcess)[2:-3])
    except Exception as e:
        return str(e)
    
readAsset('asset1')    

{'ID': 'asset1',
 'color': 'blue',
 'size': 5,
 'owner': 'Tomoko',
 'appraisedValue': 300}

In [13]:
readAsset('asset2')  

{'ID': 'asset2',
 'color': 'red',
 'size': 5,
 'owner': 'Brad',
 'appraisedValue': 400}

In [15]:
readAsset('asset4')  

{'ID': 'asset4',
 'color': 'yellow',
 'size': 10,
 'owner': 'Max',
 'appraisedValue': 600}

In [ ]:
!peer chaincode query -C mychannel -n basic -c '{"function":"GetAllAssets","Args":[]}'

# REST

In [ ]:
!peer channel fetch config /tmp/config_block.pb -o orderer.example.com:7050 -c $CHANNEL_NAME --tls --cafile $ORDERER_CA > /dev/null  

In [ ]:
#peer channel fetch config config_block.pb -o orderer.example.com:7050 -c $CHANNEL_NAME --tls --cafile $ORDERER_CA
import sys    
def channelFetch(channelName,ordererpem):
    #rint(channelName,ordererpem)
    try:    
        output = subprocess.check_output(['peer', 'channel','fetch', 'config', 'config_block.pb', '-o', 'orderer.example.com:7050', '-c',channelName, '--tls', 
                                               '--cafile', ordererpem], stderr=subprocess.STDOUT, shell=True, timeout=3,universal_newlines=True)
        #output = subprocess.check_output(['peer', 'channel','fetch', 'config', 'config_block.pb', '-o', 'orderer.example.com:7050', '-c',channelName, '--tls', 'cafile', ordererpem], stderr=subprocess.STDOUT, shell=True, timeout=3,universal_newlines=True)
        #output = subprocess.check_output(['peer', 'channel','fetch','newest', 'config', '/tmp/config_block.pb', '-o', 'orderer.example.com:7050', '-c',channelName, '--tls', '--cafile', ordererpem], stderr=subprocess.STDOUT)
        
    #xcept Exception as e:
    #   print('exception' + str(e))
    #   return None
    except subprocess.CalledProcessError as exc:
        print("Status : FAIL", exc.returncode, exc.output)
    else:
        print("Output: \n{}\n".format(output))
    
#channelFetch('mychannel',ordererpem)    

#!cd /home/jovyan/work/
#rm -f config_block.pb
channelFetch('mychannel',ordererpem)
